# 📊 Análise Exploratória de Dados (EDA)

Este notebook realiza uma análise exploratória dos dados coletados do X/Twitter e Reddit para análise de sentimento.

## Objetivos:
- 🔍 Inspeção visual dos dados antes do modelo
- 📈 Detectar outliers e viés linguístico
- 📊 Análise de distribuições e padrões
- 🖼️ Gerar visualizações para o artigo

In [ ]:
# Imports e configuração
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Criar diretório de figuras se não existir
import os
os.makedirs('../figures', exist_ok=True)

print("📦 Bibliotecas carregadas com sucesso!")

In [ ]:
# Carregar dados coletados
import glob
from pathlib import Path

# Buscar arquivos de dados
data_files = glob.glob('../data/raw/*.json') + glob.glob('../data/raw/*.csv')

if not data_files:
    print("❌ Nenhum arquivo encontrado em data/raw/")
    print("🔧 Execute primeiro o notebook 01_coleta.ipynb")
else:
    print(f"📁 Arquivos encontrados: {len(data_files)}")
    for file in data_files:
        print(f"   📄 {Path(file).name}")

# Carregar dados (assumindo formato padrão)
dfs = []
for file in data_files:
    try:
        if file.endswith('.json'):
            df_temp = pd.read_json(file, lines=True)
        else:
            df_temp = pd.read_csv(file)
        
        # Adicionar coluna de origem
        df_temp['file_source'] = Path(file).stem
        dfs.append(df_temp)
        print(f"✅ {Path(file).name}: {len(df_temp)} registros")
    except Exception as e:
        print(f"❌ Erro ao carregar {file}: {e}")

if dfs:
    # Combinar todos os DataFrames
    df = pd.concat(dfs, ignore_index=True)
    print(f"\n📊 Dataset combinado: {len(df)} registros")
else:
    print("❌ Nenhum dado carregado")

In [ ]:
# Inspeção inicial dos dados
print("🔍 INSPEÇÃO INICIAL DOS DADOS")
print("=" * 40)

# Informações básicas
print(f"📏 Shape: {df.shape}")
print(f"📊 Colunas: {list(df.columns)}")
print(f"💾 Memória: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Primeiras linhas
print("\n👀 Primeiras 3 linhas:")
display(df.head(3))

# Informações sobre tipos e valores nulos
print("\n📋 Informações sobre as colunas:")
df.info()

In [ ]:
# Identificar colunas de texto principal
text_columns = []
possible_text_cols = ['text', 'content', 'body', 'title', 'tweet', 'post']

for col in df.columns:
    if any(keyword in col.lower() for keyword in possible_text_cols):
        text_columns.append(col)

if not text_columns:
    # Tentar detectar automaticamente
    for col in df.columns:
        if df[col].dtype == 'object' and df[col].str.len().mean() > 20:
            text_columns.append(col)

print(f"📝 Colunas de texto identificadas: {text_columns}")

# Usar a primeira coluna de texto como principal
if text_columns:
    main_text_col = text_columns[0]
    print(f"🎯 Coluna principal: {main_text_col}")
    
    # Adicionar coluna de comprimento do texto
    df['text_length'] = df[main_text_col].astype(str).str.len()
    
    print(f"📏 Estatísticas de comprimento:")
    print(df['text_length'].describe())
else:
    print("❌ Nenhuma coluna de texto encontrada")

In [ ]:
# 1. HISTOGRAMA DE TAMANHO DOS TEXTOS
if text_columns:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Histograma geral
    ax1.hist(df['text_length'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
    ax1.set_xlabel('Comprimento do Texto (caracteres)')
    ax1.set_ylabel('Frequência')
    ax1.set_title('Distribuição do Comprimento dos Textos')
    ax1.grid(True, alpha=0.3)
    
    # Box plot por plataforma (se existir coluna de plataforma)
    platform_cols = ['platform', 'source', 'file_source']
    platform_col = None
    
    for col in platform_cols:
        if col in df.columns:
            platform_col = col
            break
    
    if platform_col:
        df.boxplot(column='text_length', by=platform_col, ax=ax2)
        ax2.set_title('Comprimento por Plataforma')
        ax2.set_ylabel('Comprimento do Texto')
    else:
        # Box plot geral
        ax2.boxplot(df['text_length'])
        ax2.set_title('Box Plot - Comprimento dos Textos')
        ax2.set_ylabel('Comprimento do Texto')
    
    plt.tight_layout()
    plt.savefig('../figures/text_length_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("💾 Figura salva: figures/text_length_distribution.png")

In [ ]:
# 2. WORDCLOUD DOS TEXTOS
if text_columns:
    # Combinar todos os textos
    all_text = ' '.join(df[main_text_col].astype(str).tolist())
    
    # Criar WordCloud
    wordcloud = WordCloud(
        width=800, 
        height=400, 
        background_color='white',
        max_words=100,
        colormap='viridis',
        relative_scaling=0.5
    ).generate(all_text)
    
    plt.figure(figsize=(12, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('Word Cloud - Termos Mais Frequentes', fontsize=16, pad=20)
    
    plt.savefig('../figures/wordcloud.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("💾 Figura salva: figures/wordcloud.png")

In [ ]:
# 3. ANÁLISE TEMPORAL
# Tentar identificar coluna de data
date_columns = []
possible_date_cols = ['date', 'created_at', 'timestamp', 'time']

for col in df.columns:
    if any(keyword in col.lower() for keyword in possible_date_cols):
        date_columns.append(col)

print(f"📅 Colunas de data identificadas: {date_columns}")

if date_columns:
    date_col = date_columns[0]
    
    # Converter para datetime se necessário
    try:
        df[date_col] = pd.to_datetime(df[date_col])
        
        # Criar coluna de data (sem hora)
        df['date_only'] = df[date_col].dt.date
        
        # Análise temporal por dia
        daily_counts = df.groupby('date_only').size().reset_index(name='count')
        
        plt.figure(figsize=(12, 6))
        plt.plot(daily_counts['date_only'], daily_counts['count'], marker='o', linewidth=2)
        plt.xlabel('Data')
        plt.ylabel('Número de Posts')
        plt.title('Distribuição Temporal dos Posts')
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('../figures/temporal_distribution.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print("💾 Figura salva: figures/temporal_distribution.png")
        
        # Estatísticas temporais
        print(f"📊 Período: {df[date_col].min()} até {df[date_col].max()}")
        print(f"📅 Dias únicos: {df['date_only'].nunique()}")
        print(f"📈 Média posts/dia: {daily_counts['count'].mean():.1f}")
        
    except Exception as e:
        print(f"❌ Erro ao processar datas: {e}")
else:
    print("⚠️ Nenhuma coluna de data encontrada")

In [ ]:
# 4. DETECÇÃO DE IDIOMA
try:
    from langdetect import detect
    
    print("🔄 Detectando idiomas...")
    
    # Função para detectar idioma de forma segura
    def safe_detect_lang(text):
        try:
            return detect(str(text))
        except:
            return 'unknown'
    
    # Detectar idioma em uma amostra (para ser mais rápido)
    sample_size = min(1000, len(df))
    df_sample = df.sample(n=sample_size, random_state=42)
    df_sample['detected_lang'] = df_sample[main_text_col].apply(safe_detect_lang)
    
    # Contar idiomas
    lang_counts = df_sample['detected_lang'].value_counts()
    
    # Gráfico de pizza
    plt.figure(figsize=(10, 8))
    
    # Mostrar apenas os 6 idiomas mais comuns
    top_langs = lang_counts.head(6)
    others = lang_counts.iloc[6:].sum()
    
    if others > 0:
        top_langs['outros'] = others
    
    colors = plt.cm.Set3(np.linspace(0, 1, len(top_langs)))
    wedges, texts, autotexts = plt.pie(top_langs.values, 
                                       labels=top_langs.index, 
                                       autopct='%1.1f%%',
                                       colors=colors,
                                       startangle=90)
    
    plt.title(f'Distribuição de Idiomas (amostra de {sample_size} posts)', fontsize=14)
    
    plt.savefig('../figures/language_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Estatísticas
    pt_percentage = (lang_counts.get('pt', 0) / sample_size) * 100
    print(f"🇧🇷 Português: {pt_percentage:.1f}% dos textos")
    print(f"🌍 Total de idiomas detectados: {len(lang_counts)}")
    
    print("💾 Figura salva: figures/language_distribution.png")
    
except ImportError:
    print("⚠️ langdetect não instalado. Execute: pip install langdetect")
except Exception as e:
    print(f"❌ Erro na detecção de idiomas: {e}")

In [ ]:
# 5. DISTRIBUIÇÃO POR PLATAFORMA
if platform_col:
    platform_counts = df[platform_col].value_counts()
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Gráfico de barras
    platform_counts.plot(kind='bar', ax=ax1, color='lightcoral')
    ax1.set_title('Posts por Plataforma')
    ax1.set_ylabel('Número de Posts')
    ax1.tick_params(axis='x', rotation=45)
    
    # Gráfico de pizza
    ax2.pie(platform_counts.values, labels=platform_counts.index, autopct='%1.1f%%')
    ax2.set_title('Distribuição por Plataforma')
    
    plt.tight_layout()
    plt.savefig('../figures/platform_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("💾 Figura salva: figures/platform_distribution.png")
    
    # Estatísticas por plataforma
    print("\n📊 Estatísticas por plataforma:")
    for platform in platform_counts.index:
        platform_data = df[df[platform_col] == platform]
        avg_length = platform_data['text_length'].mean()
        print(f"   📱 {platform}: {len(platform_data)} posts, média {avg_length:.0f} chars")
else:
    print("⚠️ Coluna de plataforma não identificada")

In [ ]:
# 6. ANÁLISE DE OUTLIERS
if text_columns:
    print("🔍 ANÁLISE DE OUTLIERS")
    print("=" * 30)
    
    # Outliers por comprimento
    Q1 = df['text_length'].quantile(0.25)
    Q3 = df['text_length'].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df['text_length'] < lower_bound) | (df['text_length'] > upper_bound)]
    
    print(f"📏 Outliers por comprimento: {len(outliers)} ({len(outliers)/len(df)*100:.1f}%)")
    
    # Textos muito curtos
    very_short = df[df['text_length'] < 10]
    print(f"📝 Textos muito curtos (<10 chars): {len(very_short)}")
    
    # Textos muito longos
    very_long = df[df['text_length'] > 1000]
    print(f"📖 Textos muito longos (>1000 chars): {len(very_long)}")
    
    # Mostrar exemplos
    if len(very_short) > 0:
        print("\n🔍 Exemplos de textos muito curtos:")
        for i, text in enumerate(very_short[main_text_col].head(3)):
            print(f"   {i+1}. '{text}' ({len(str(text))} chars)")
    
    if len(very_long) > 0:
        print("\n🔍 Exemplos de textos muito longos:")
        for i, text in enumerate(very_long[main_text_col].head(2)):
            print(f"   {i+1}. '{str(text)[:100]}...' ({len(str(text))} chars)")

In [ ]:
# 7. RELATÓRIO FINAL E EXPORTAÇÃO
print("📋 RELATÓRIO FINAL DA EDA")
print("=" * 35)

# Resumo geral
total_posts = len(df)
avg_length = df['text_length'].mean() if 'text_length' in df.columns else 0
unique_dates = df['date_only'].nunique() if 'date_only' in df.columns else 'N/A'
platforms = df[platform_col].nunique() if platform_col else 'N/A'

summary_stats = {
    'total_posts': total_posts,
    'avg_text_length': avg_length,
    'unique_dates': unique_dates,
    'platforms': platforms,
    'memory_usage_mb': df.memory_usage(deep=True).sum() / 1024**2
}

print(f"📊 Total de posts: {total_posts:,}")
print(f"📏 Comprimento médio: {avg_length:.0f} caracteres")
print(f"📅 Dias únicos: {unique_dates}")
print(f"📱 Plataformas: {platforms}")
print(f"💾 Uso de memória: {summary_stats['memory_usage_mb']:.2f} MB")

# Salvar estatísticas resumidas
import json
os.makedirs('../results', exist_ok=True)

with open('../results/eda_summary.json', 'w', encoding='utf-8') as f:
    json.dump(summary_stats, f, indent=2, default=str)

print("\n💾 Estatísticas salvas em: results/eda_summary.json")

# Lista de figuras geradas
figures_generated = [
    'text_length_distribution.png',
    'wordcloud.png',
    'temporal_distribution.png',
    'language_distribution.png',
    'platform_distribution.png'
]

print("\n🎨 Figuras geradas:")
for fig in figures_generated:
    fig_path = f"../figures/{fig}"
    if os.path.exists(fig_path):
        print(f"   ✅ {fig}")
    else:
        print(f"   ❌ {fig} (não encontrada)")

print("\n🎉 EDA Completa! Dados prontos para rotulagem e modelagem.")